In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import requests
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import os
import re

In [2]:
## access token informations 
access_token1 = "1222829174802538496-AXkAccE1dWMjuEFlEYChpgOibc6SbF"
access_token_secret1 = "jtr66CvVvspsANplSBpuMEQR5iLwK2fRtzM6aDhaC1rZT"

consumer_key1 = "sL9E5QL83DVEshfttLjEEj930"
consumer_secret1 = "R9L4Ar7UWTKB4WQw7cEoybumyTInZ6pYP3cQ5GFlWYFllYAeec"

In [3]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [51]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [52]:
OUTPUT_FILENAME = "other-outputs/tweets_US_Election_2020.json"
stop_condition = 500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["Donald", "Trump", "Joe", "Biden", "America", "USA"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

In [53]:
json_election_file = []
for line in open(OUTPUT_FILENAME, 'r'):
    json_election_file.append(json.loads(line))

In [54]:
dict_tweets={}
i=0
for tweet in json_election_file:
    dict_tweets[i]=tweet
    i+=1
tweets = pd.DataFrame.from_dict(dict_tweets, orient='index')
tweets.reset_index(level=0, inplace=False)
tweets.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,possibly_sensitive,display_text_range,extended_tweet,extended_entities
0,Mon Nov 30 12:19:06 +0000 2020,1333385061723324419,1333385061723324419,"RT @briebriejoy: Are ""hunger and nutrition"" no...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 1204555554586128384, 'id_str': '1204555...",None,None,None,None,{'created_at': 'Mon Nov 30 04:58:09 +0000 2020...,1.333245e+18,1333245132964061184,{'created_at': 'Mon Nov 30 03:03:04 +0000 2020...,"{'url': 'https://t.co/RMJd0IeVXf', 'expanded':...",True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746468,NaN,NaN,NaN,NaN
1,Mon Nov 30 12:19:06 +0000 2020,1333385061719298049,1333385061719298049,"It is fair, and urgent. Esp. in KY, SC and Maine.","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,None,"{'id': 627341141, 'id_str': '627341141', 'name...",None,None,None,None,NaN,1.333260e+18,1333259746191810561,{'created_at': 'Mon Nov 30 04:01:08 +0000 2020...,"{'url': 'https://t.co/tvUbeTnVK5', 'expanded':...",True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746467,NaN,NaN,NaN,NaN
2,Mon Nov 30 12:19:06 +0000 2020,1333385061970837504,1333385061970837504,RT @Rasmussen_Poll: Almost half (47%) of U.S. ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 3310664641, 'id_str': '3310664641', 'na...",None,None,None,None,{'created_at': 'Sun Nov 29 18:20:12 +0000 2020...,1.329742e+18,1329741803025801217,{'created_at': 'Fri Nov 20 11:02:05 +0000 2020...,"{'url': 'https://t.co/7BLJLfLQaJ', 'expanded':...",True,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746527,NaN,NaN,NaN,NaN
3,Mon Nov 30 12:19:06 +0000 2020,1333385062155513857,1333385062155513857,RT @Goss30Goss: When Trump is convicted &amp; ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,None,"{'id': 4081545435, 'id_str': '4081545435', 'na...",None,None,None,None,{'created_at': 'Mon Nov 30 03:18:51 +0000 2020...,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746571,NaN,NaN,NaN,NaN
4,Mon Nov 30 12:19:06 +0000 2020,1333385062193057799,1333385062193057799,"literally ""fall""\nhttps://t.co/uij0ZiPde0","<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 1331842561439199233, 'id_str': '1331842...",None,None,None,None,NaN,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1606738746580,False,NaN,NaN,NaN
5,Mon Nov 30 12:19:06 +0000 2020,1333385062176468993,1333385062176468993,@_Mercian @realDonaldTrump Framed another way....,"<a href=""http://twitter.com/download/android"" ...",True,1.332471e+18,1332471058360918017,1.272608e+18,1272607681757446145,_Mercian,"{'id': 3439131881, 'id_str': '3439131881', 'na...",None,None,None,None,NaN,NaN,NaN,NaN,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1606738746576,NaN,"[27, 140]",{'full_text': '@_Mercian @realDonaldTrump Fram...,NaN
6,Mon Nov 30 12:19:06 +0000 2020,1333385062029651969,1333385062029651969,RT @MollieKatzen: If you're not already suppor...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,None,"{'id': 14919424, 'id_str': '14919424', 'name':...",None,None,None,None,{'created_at': 'Sat Nov 28 16:38:36 +0000 2020...,1.331622e+18,1331622040353300480,{'created_at': 'Wed Nov 25 15:33:29 +0000 2020...,"{'url': 'https:

In [55]:
pd.set_option('display.max_columns', None)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1002 entries, 0 to 1001
Data columns (total 36 columns):
created_at                   1002 non-null object
id                           1002 non-null int64
id_str                       1002 non-null object
text                         1002 non-null object
source                       1002 non-null object
truncated                    1002 non-null bool
in_reply_to_status_id        151 non-null float64
in_reply_to_status_id_str    151 non-null object
in_reply_to_user_id          159 non-null float64
in_reply_to_user_id_str      159 non-null object
in_reply_to_screen_name      159 non-null object
user                         1002 non-null object
geo                          0 non-null object
coordinates                  0 non-null object
place                        8 non-null object
contributors                 0 non-null object
retweeted_status             694 non-null object
quoted_status_id             180 non-null float64
quoted_statu

In [56]:
#Normalize functions:

#NO SALEN HASHTAGS NI DEL TEXT NI DE ENTITIES.MIRAR RETWEETED STATUS Y TIENE QUE DEVOLVER
def get_hashtags(data: pd.DataFrame) -> None:
    hashtag_list=[]
    for i in range(len(data)):
        hashtags=[]
        try:
            hashtags_info = data['entities']['hashtags']
        except:
            hashtags_info = data['entities'][i]['hashtags']
        for i in range (len(hashtags_info)):
            hashtags.append(hashtags_info[i]["text"])
        hashtag_list.append(hashtags)
    data['hashtags']=hashtag_list

def remove_unnecessary_columns(data: pd.DataFrame)-> pd.DataFrame:
    cols_to_remove=['id', 'display_text_range', 'in_reply_to_status_id', 
                    'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'geo', 'coordinates', 
                    'place', 'contributors', 'is_quote_status', 'quote_count','lang', 
                    'possibly_sensitive', 'quoted_status_id','quoted_status_id_str', 'quoted_status', 
                    'quoted_status_permalink']
    
    data=data.drop(columns=cols_to_remove)
    return data

#retweeted_status only exists if the tweet has been retweeted by someone. If not, extended_tweet exists if Truncated is true.
#If both are not the case, we use directly the information given in the tweet.
def get_tweet_data(data: pd.DataFrame) -> pd.DataFrame:
    #  Tweet| Username | Date | Hashtags | Likes | Retweets | Url
    tweets_text =[]
    tweets_username =[]
    tweets_date = []
    tweets_hashtags = []
    tweets_likes = []
    tweets_retweets = []
    tweets_url = []
    for tweet in range(len(data)):
        
        if str(data.iloc[tweet]['retweeted_status'])=='nan':
            if(data["truncated"][tweet] == False):
                tweets_text.append(data['text'][tweet])
                tweets_hashtags.append(get_hashtags(data))
            else:
                tweets_text.append(data['extended_tweet'][tweet]['full_text'])
                tweets_hashtags.append(get_hashtags(data['extended_tweet'][tweet]))
            
            tweets_date.append(data['created_at'][tweet])
            tweets_username.append(data['user'][tweet]['name'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
            tweets_likes.append(data['favorite_count'][tweet])
            tweets_retweets.append(data['retweet_count'][tweet])
        else:
            tweets_text.append(data['retweeted_status'][tweet]['text'])
            #We supposed it is the name and not the @name (screen_name)
            tweets_username.append(data['retweeted_status'][tweet]['user']['name']) 
            tweets_date.append(data['retweeted_status'][tweet]['created_at'])
            tweets_hashtags.append(get_hashtags((data['retweeted_status'][tweet])))
            tweets_likes.append(data['retweeted_status'][tweet]['favorite_count'])
            tweets_retweets.append(data['retweeted_status'][tweet]['retweet_count'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
        
    tweet_info = {'Tweet': tweets_text, 'Username': tweets_username, 'Date': tweets_date, 'Hashtags': tweets_hashtags, 
         'Likes': tweets_likes, 'Retweets': tweets_retweets, 'Url': tweets_url}

    return pd.DataFrame(data=tweet_info)

In [57]:
def decontracted(text: str) -> str:
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def punctuation_removal(text: str) -> str:
    tokenizer=RegexpTokenizer(r'\w+')
    print(tokenizer)
    return tokenizer.tokenize(decontracted(text.lower()))

def stemming(text: list) -> list:
    ps = PorterStemmer()
    stemmed_text = []
    for word in text: 
        stemmed_text.append(ps.stem(word))
    return stemmed_text

def remove_stop_words(word_list: list) -> list:
    stop_words = set(stopwords.words('english'))
    stop_words.add('rt')
    text_without_stopwords = []
    for word in word_list:
        if not word in stop_words:
            text_without_stopwords.append(word)
    return text_without_stopwords

def text_normalization(text: str) -> str:
    text_normalized = punctuation_removal(text)
    text_stemmed  = stemming(text_normalized)
    text_without_stopwords = remove_stop_words(text_stemmed)
    return " ".join(text_without_stopwords)

In [58]:
#Test to get out rt:
#print(text_normalization('RT, Toca.Ovaris, 20.000 or 20,00'))

In [69]:
tweets=remove_unnecessary_columns(tweets)
tweets.head()

KeyError: "['id' 'display_text_range' 'in_reply_to_status_id'\n 'in_reply_to_status_id_str' 'in_reply_to_user_id'\n 'in_reply_to_user_id_str' 'geo' 'coordinates' 'place' 'contributors'\n 'is_quote_status' 'quote_count' 'lang' 'possibly_sensitive'\n 'quoted_status_id' 'quoted_status_id_str' 'quoted_status'\n 'quoted_status_permalink'] not found in axis"

In [60]:
def inverted_index(data: pd.DataFrame)->dict:
    inv_index={}
    for doc_num in range(len(data['text'])):
        words=data['text'][doc_num].split()
        for word in words:
            if word in inv_index:
                if word not in inv_index[word]:
                    inv_index[word].append(doc_num)
            else:
                inv_index[word]=[doc_num]
    return inv_index

#REMOVE THIS FUNCTION, THERE MUST BE A LIBRARY THAT DOES THIS
def tf_idf_normalization(tf_idf: list) -> None:
    D_square=np.zeros(len(tf_idf))
    for i in range(len(tf_idf)):
        document_values=list(tf_idf[i].values())
        for j in range(len(document_values)):
            document_values[j]=(document_values[j]**2)
        D_square[i] = np.sum(document_values)
    
    for i in range (len(tf_idf)):
        for word in tf_idf[i]:
            tf_idf[i][word]= tf_idf[i][word]/np.sqrt(D_square[i])

def tf_idf(docs: list, inv_index: dict) -> list:
    tf=[]
    df={}
    idf={}
    tf_idf=[]
    N=len(tweets)
    for doc_text in docs:
        tf_doc_i={}
        words = doc_text.split()
        for word in words:
            if word in tf_doc_i:
                tf_doc_i[word]+=1/len(words)
            else:
                tf_doc_i[word]=1/len(words)
        tf.append(tf_doc_i)
        
    for word in inv_index:
        df[word]=len(inv_index[word])
        idf[word]=np.log(float(N/(df[word]+1)))

    for doc in range(len(docs)):
        tf_idf_per_doc={}
        for word in tf[doc]:
            tf_idf_per_doc[word]=tf[doc][word]*idf[word]
        tf_idf.append(tf_idf_per_doc)
        
    tf_idf_normalization(tf_idf)
    return tf_idf


In [61]:
def query_tfidf(query: str, query_vector: dict) -> None:
    query_tf_idf = tf_idf([query], inverted_index(tweets))
    for word in query.split():
        query_vector[word]= query_tf_idf[0][word]
    
def ranking_documents(query: str, tfidf: list):
    cosine_similarity={}
    i=0
    for document in tfidf:
        vectorized_doc = list(document.values())
        if all(word in document.keys() for word in query.split()):
            #THIS CAN BE IMPROVED
            vectorized_query = {key:0 for key in document.keys()}
            query_tfidf(query, vectorized_query)
            vectorized_query_values = list(vectorized_query.values())
            cosine_similarity[i] = np.dot(vectorized_query_values,vectorized_doc)
        else:
            cosine_similarity[i] = 0
        i+=1
    return cosine_similarity

In [62]:
database=get_tweet_data(tweets)
for i in range(len(tweets)):
    tweets['text'][i]=text_normalization(tweets['text'][i])

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)


C:\Users\TESTER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILIN

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILIN

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILIN

...

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILIN

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILIN

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILIN

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILIN

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILIN

In [63]:
database

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,"Are ""hunger and nutrition"" not issues in rural...",BJG 👑,Mon Nov 30 04:58:09 +0000 2020,None,2065,204,https://twitter.com/i/web/status/1333385061723...
1,"It is fair, and urgent. Esp. in KY, SC and Maine.",JHWBB,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385061719...
2,Almost half (47%) of U.S. likley voters now be...,Rasmussen Reports,Sun Nov 29 18:20:12 +0000 2020,None,16993,12098,https://twitter.com/i/web/status/1333385061970...
3,"When Trump is convicted &amp; imprisonedin NY,...",Andrew Goss 👊USAF👊,Mon Nov 30 03:18:51 +0000 2020,None,1781,202,https://twitter.com/i/web/status/1333385062155...
4,"literally ""fall""\nhttps://t.co/uij0ZiPde0",パンケーキ,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062193...
5,@_Mercian @realDonaldTrump Framed another way....,cinda collins,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062176...
6,If you're not already supporting @gaslitnation...,Mollie Katzen,Sat Nov 28 16:38:36 +0000 2020,None,159,38,https://twitter.com/i/web/status/1333385062029...
7,Today’s observations:\nBiden did not break his...,Warrior on Krak,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062440...
8,"So, we wake up to see that #StopTheStupid is t...",︻╦̵̵͇̿̿̿̿ Justice Johnson╤,Mon Nov 30 11:12:12 +0000 2020,None,66,22,https://twitter.com/i/web/status/1333385062583...
9,I will never give up on Trump.,LORI HENDRY,Mon Nov 30 00:50:59 +0000 2020,None,7200,1190,https://twitter.com/i/web/status/1333385062679...


In [64]:
tweets["text"].tolist()
tfidf=tf_idf(tweets["text"].tolist(), inverted_index(tweets))

In [65]:
print("Insert your query:\n")
query = input()
query=text_normalization(query)
docs = ranking_documents(query, tfidf)    
top = 20

database["score"]= pd.DataFrame(np.zeros(len(database)))
database["score"]=docs.values()
database_sorted=database.sort_values(by=["score"], ascending=False)
database_sorted.head(top)

#CUBRIR LOS CASOS DONDE NO SE LLEGA AL TOP 20 BUSCANDO MAS TWEETS
#Creo que ni la score ni gd_score tendrian que salir en el output (database), sinó en tweets database. 

Insert your query:

joe biden
RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)


,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,score
498,Joe Biden will be president in 51 days.,Kyle Griffin,Mon Nov 30 12:00:00 +0000 2020,None,2231,165,https://twitter.com/i/web/status/1333385115574...,0.333017
647,Joe Biden will be president in 51 days.,Kyle Griffin,Mon Nov 30 12:00:00 +0000 2020,None,4553,316,https://twitter.com/i/web/status/1333392720178...,0.333017
493,Joe Biden\nJosiah Bartlet,hi,Mon Nov 30 12:19:19 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385115360...,0.325277
50,Joe👏Biden👏is👏not👏more👏popular👏than👏Barack👏Obama👏,Lauren Boebert,Mon Nov 30 12:06:56 +0000 2020,None,497,82,https://twitter.com/i/web/status/1333385068312...,0.292700
10,Joe Biden got injured playing with his massive...,BrooklynDad_Defiant!,Mon Nov 30 02:09:39 +0000 2020,None,15710,1978,https://twitter.com/i/web/status/1333385062688...,0.258200
430,Joe Biden got injured playing with his massive...,BrooklynDad_Defiant!,Mon Nov 30 02:09:39 +0000 2020,None,15713,1979,https://twitter.com/i/web/status/1333385107252...,0.258200
359,Joe Biden twisting his ankle while playing wit...,Julian's Rum 🥃,Sun Nov 29 22:06:02 +0000 2020,None,10795,1584,https://twitter.com/i/web/status/1333385100571...,0.255293
450,I don't believe for one second that Joe Biden ...,Patriot Jen🇺🇸,Mon Nov 30 12:12:11 +0000 2020,None,24,4,https://twitter.com/i/web/status/1333385109345...,0.252804
15,Joe👏Biden👏is👏not👏more👏popular👏than👏Barack👏Obam...,Lauren Boebert,Mon Nov 30 12:06:56 +0000 2020,None,496,81,https://twitter.com/i/web/status/1333385062931...,0.248778
658,. whats joe bidens plan for abolishing The e...,lia 🐇,Mon Nov 30 12:49:32 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333392721076...,0.245023


In [66]:
# get always full text by checking if truncated. If it is, then find extended_tweet or retweeted_status.

In [67]:
def g_d_score(data: pd.DataFrame) -> list: #Hay que pasar tweets database
    #retweets -> 3/6; likes -> 2/6; replies -> 1/6
    gd_score=[]
    for row in data.iterrows():
        retweets_score=row[1]['retweet_count']
        print(retweets_score)
        likes_score=row[1]['favorite_count']
        print(likes_score)
        replies_score=row[1]['reply_count']
        print(replies_score)
        score=((3/6)*int(retweets_score)+(2/6)*int(likes_score)+(1/6)*int(replies_score))
        print(score)
        gd_score.append(score)
        print(gd_score)
    return gd_score

In [68]:
database["g(d)"]= pd.DataFrame(np.zeros(len(database)))
l=g_d_score(tweets)

#for row in tweets.iterrows:
#    print(row)
    #print(row[1]['retweet_count'])
    #print()
tweets

0
0
0
0.0
[0.0]
0
0
0
0.0
[0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

...

0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

0
0
0
0.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

,created_at,id_str,text,source,truncated,in_reply_to_screen_name,user,retweeted_status,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,timestamp_ms,extended_tweet,extended_entities,hashtags
0,Mon Nov 30 12:19:06 +0000 2020,1333385061723324419,briebriejoy hunger nutrit issu rural america,"<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'id': 1204555554586128384, 'id_str': '1204555...",{'created_at': 'Mon Nov 30 04:58:09 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746468,NaN,NaN,[]
1,Mon Nov 30 12:19:06 +0000 2020,1333385061719298049,fair urgent esp ky sc main,"<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'id': 627341141, 'id_str': '627341141', 'name...",NaN,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746467,NaN,NaN,[]
2,Mon Nov 30 12:19:06 +0000 2020,1333385061970837504,rasmussen_pol almost half 47 u likley voter be...,"<a href=""http://twitter.com/download/android"" ...",False,None,"{'id': 3310664641, 'id_str': '3310664641', 'na...",{'created_at': 'Sun Nov 29 18:20:12 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746527,NaN,NaN,[]
3,Mon Nov 30 12:19:06 +0000 2020,1333385062155513857,goss30goss trump convict amp imprisonedin ny o...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,"{'id': 4081545435, 'id_str': '4081545435', 'na...",{'created_at': 'Mon Nov 30 03:18:51 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746571,NaN,NaN,[]
4,Mon Nov 30 12:19:06 +0000 2020,1333385062193057799,liter fall http co uij0zipde0,"<a href=""http://twitter.com/download/android"" ...",False,None,"{'id': 1331842561439199233, 'id_str': '1331842...",NaN,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,1606738746580,NaN,NaN,[]
5,Mon Nov 30 12:19:06 +0000 2020,1333385062176468993,_mercian realdonaldtrump frame anoth way eight...,"<a href=""http://twitter.com/download/android"" ...",True,_Mercian,"{'id': 3439131881, 'id_str': '3439131881', 'na...",NaN,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,1606738746576,{'full_text': '@_Mercian @realDonaldTrump Fram...,NaN,[]
6,Mon Nov 30 12:19:06 +0000 2020,1333385062029651969,molliekatzen alreadi support gaslitn patreon p...,"<a href=""http://twitter.com/download/android"" ...",False,None,"{'id': 14919424, 'id_str': '14919424', 'name':...",{'created_at': 'Sat Nov 28 16:38:36 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746541,NaN,NaN,[]
7,Mon Nov 30 12:19:06 +0000 2020,1333385062440726531,today observ biden break hi foot democrat real...,"<a href=""http://twitter.com/download/iphone"" r...",True,None,"{'id': 1445888340, 'id_str': '1445888340', 'na...",NaN,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,1606738746639,{'full_text': 'Today’s observations: Biden did...,NaN,[]
8,Mon Nov 30 12:19:06 +0000 2020,1333385062583324687,acjjustic wake see stopthestupid trend know wi...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'id': 1089125428017676289, 'id_str': '1089125...",{'created_at': 'Mon Nov 30 11:12:12 +0000 2020...,0,0,0,"{'hashtags': [{'text': 'StopTheStupid', 'indic...",False,False,low,1606738746673,NaN,NaN,[StopTheStupid]
9,Mon Nov 30 12:19:06 +0000 2020,1333385062679764997,lrihendri never give trump,"<a href=""http://twitter.com/download/android"" ...",False,None,"{'id': 1029096279685648385, 'id_str': '1029096...",{'created_at': 'Mon Nov 30 00:50:59 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746696,NaN,NaN,[]


In [ ]:
os.remove(OUTPUT_FILENAME)

In [ ]:
#Quitar URLs
#Quitar emojis
#CLEAN .JSON SO IT DOES NOT SUM TWEETS IF RUNED AGAIN